### Sentencepiece : NSMC 데이터로 tokenizer 구성해보기
: https://github.com/google/sentencepiece  

내부 단어 분리를 위한 유용한 패키지: 구글의 센텐스피스(Sentencepiece)

In [1]:
! pip install sentencepiece

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import sentencepiece as spm
import csv

In [16]:
# 데이터 읽어오기 : ratings_train.txt

train_df = pd.read_csv('data/ratings_train.csv')
naver_df = pd.read_table('data/ratings_train.txt')

In [13]:
# Null 값 제거 / 확인
train_df.isnull().sum()
train_df=train_df.dropna()
train_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [19]:
naver_df = naver_df.dropna(how='any')
print(naver_df.isnull().sum())

id          0
document    0
label       0
dtype: int64


In [21]:
# 최종적으로 전처리된 텍스트를 'naver_review.txt'에 저장 (문장 \n 구분)
print(len(naver_df))
with open('./data/naver_review.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(naver_df['document']))

f.close()

149995


In [23]:
# 학습하기: 2개 파일 생성 .vocab (subword), .model
spm.SentencePieceTrainer.Train('--input=./data/naver_review.txt --model_prefix=naver --vocab_size=5000 --model_type=bpe --max_sentence_length=9999')

위의 코드는 SentencePiece를 사용하여 텍스트 데이터에서 Subword 기반의 언어 모델을 학습시키는 과정을 나타냅니다.

여기서 각 매개변수의 역할은 다음과 같습니다:

--input=./data/naver_review.txt: 학습에 사용할 텍스트 데이터의 경로를 지정합니다. 여기서는 "./data/naver_review.txt"에 해당합니다.
--model_prefix=naver: 생성될 모델 파일의 접두사를 지정합니다. 여기서는 "naver"로 지정되어 있으므로 학습이 완료되면 "naver.model" 및 "naver.vocab" 파일이 생성됩니다.
--vocab_size=5000: 어휘 크기를 지정합니다. 이것은 모델이 학습하는 서브워드의 수입니다. 여기서는 5000으로 지정되어 있으므로 5000개의 서브워드로 어휘를 구성하게 됩니다.
--model_type=bpe: 서브워드 분할 방법을 지정합니다. 여기서는 BPE(Byte Pair Encoding) 방법을 사용합니다.
--max_sentence_length=9999: 입력 문장의 최대 길이를 지정합니다. 여기서는 9999로 설정되어 있으므로 입력 문장의 최대 길이는 9999자입니다.
이렇게 학습된 모델은 텍스트 데이터를 서브워드로 분할하여 어휘 사전을 구축하게 됩니다. 이후 이 모델을 사용하여 텍스트 데이터를 분할하거나 인코딩할 수 있습니다.

In [24]:
# vocab 파일 확인해보기
vocab_list = pd.read_csv('./data/naver.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list[:10]

# 토큰과, 인덱스 번호
# 영화라는 단어가 어디 있냐에 따라 _가 붙음 (영화가 중간에 나오면 _가 붙음)

,0,1
0,<unk>,0
1,<s>,0
2,</s>,0
3,..,0
4,영화,-1
5,▁영화,-2
6,▁이,-3
7,▁아,-4
8,...,-5
9,ᄏᄏ,-6


In [25]:
vocab_list.sample(10)

,0,1
1195,▁프로,-1192
4456,밧,-4453
1202,▁이쁘,-1199
2067,▁앞으로,-2064
3478,년,-3475
2482,구려,-2479
4964,훠,-4961
4917,$,-4914
2703,▁열심히,-2700
3198,▁연기자,-3195


In [26]:
# 모델 로드
sp = spm.SentencePieceProcessor()
vocab_file = './data/naver.model'
sp.load(vocab_file)

True

In [31]:
# 모델을 통해 tokenizing
lines = [
    "털콩게 댄스댄스",
    "놀이동산 가고 싶어용 애옹 ㅋㅅㅋ",
    "눈물 롬곡"
]

for line in lines:
    print(line)
    print(sp.encode_as_pieces(line)) # encoding to token
    print(sp.encode_as_ids(line)) # encoding to idx
    print()


털콩게 댄스댄스
['▁', '털', '콩', '게', '▁', '댄', '스', '댄', '스']
[3288, 4249, 3918, 3313, 3288, 4212, 3325, 4212, 3325]

놀이동산 가고 싶어용 애옹 ㅋㅅㅋ
['▁놀', '이', '동', '산', '▁가', '고', '▁싶어', '용', '▁애', '옹', '▁ᄏ', 'ᄉ', 'ᄏ']
[577, 3290, 3384, 3659, 46, 3293, 3042, 3418, 144, 4134, 448, 3887, 3309]

눈물 롬곡
['▁눈물', '▁', '롬', '곡']
[430, 3288, 4655, 3977]



In [32]:
# 모델로 voca size 확인하기
sp.GetPieceSize()

5000

In [34]:
# id to subword
sp.IdToPiece(4249)

'털'

In [40]:
# subword to id
sp.PieceToId('털')

4249

In [43]:
# ids to subwords
sp.DecodeIds([3288, 4249, 3918, 3313, 3288, 4212, 3325, 4212, 3325])

'털콩게 댄스댄스'

In [44]:
# subwords to 원형
sp.DecodePieces(['▁', '털', '콩', '게', '▁', '댄', '스', '댄', '스'])

'털콩게 댄스댄스'

In [46]:
# encode 메소드 중 out_type 인자를 통해: subwords, ids로 변환
print(sp.encode('와웅 털콩게!', out_type=str))
print(sp.encode('와웅 털콩게!', out_type=int))

['▁와', '웅', '▁', '털', '콩', '게', '!']
[327, 3933, 3288, 4249, 3918, 3313, 3316]


토크나이징 - konlp

임베딩

토크나이저로 이미 만들어져 있어서 그냥 로드해서 쓰면 됨

llm 의 근간이 되는 모델 - 트랜스포머

내일은 트랜스포머의 구조에 대해 배우면,

버트는 왜 분류를 하는지, gpt 가 어떻게 대답을 하는지 알 수 있음

이게 왜 필요하냐? 개념적으로 중요하기 때문에 구조를 세세하게 파악하는 것이 중요